In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras import mixed_precision 
mixed_precision.set_global_policy('mixed_float16')
import geopandas as gps
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
from rasterio import windows
from rasterio import features
import fiona                     # I/O vector data (shape, geojson, ...)
import pyproj                    # Change coordinate reference system
from osgeo import gdal, ogr, osr
import pandas as pd
import shapely
from shapely.geometry import Point, Polygon,box
from shapely.geometry import mapping, shape
import cv2
import json
import geopandas as gpd
import numpy as np               # numerical array manipulation
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image
import PIL.ImageDraw
import shutil
from itertools import product

import sys
from core.UNet import UNet
from core.losses import tversky, focalTversky, bce_dice_loss, accuracy, dice_loss, IoU, recall, precision,F1_score
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.visualize import display_images

import matplotlib.pyplot as plt  # plotting tools
%matplotlib inline
import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
import tensorflow as tf
print(tf.__version__)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6
2.9.0


In [13]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [14]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/RasterAnalysis.py)
# Please provide required info in the file before continuing with this notebook. 
 
from config import RasterAnalysis_test
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import RasterAnalysis
config = RasterAnalysis_test.Configuration()

self.trained_model_path: D:\lakemapping\5_saved_models\lakes_20240108-2231_AdaDelta_dice_loss_012345_572.h5


In [5]:
# testArea = gps.read_file(r'D:\lakemapping\predition\test\test_area.shp')
# image_dir=r'G:\sample_img_3'
# output_dir=config.input_image_dir
# for root, dirs, files in os.walk(image_dir):
#         for file in files:
#             if file.endswith('.tif') and file.startswith('ndwi'):
#                 ndwiImg = rasterio.open(os.path.join(root, file))
#                 bboxImg = box(*ndwiImg.bounds)
#                 for i in tqdm(testArea.index):
#                     if(bboxImg.intersects(testArea.loc[i]['geometry'])):
#                         oldNDWI=os.path.join(root, file)
#                         newNDWI=os.path.join(output_dir,file)
#                         shutil.copy(oldNDWI, newNDWI)
#                         oldBands=oldNDWI.replace('ndwi_int8','bands_int16')
#                         newBands=newNDWI.replace('ndwi_int8','bands_int16')
#                         shutil.copy(oldBands, newBands)

In [6]:
# Load a pretrained model 
OPTIMIZER = adaDelta
# OPTIMIZER = adam
OPTIMIZER=mixed_precision.LossScaleOptimizer(OPTIMIZER)
model = load_model(config.trained_model_path, custom_objects={'dice loss': dice_loss, 'accuracy':accuracy ,'recall':recall, 'F1_score':F1_score,'precision':precision,'IoU': IoU}, compile=False)
model.compile(optimizer=OPTIMIZER, loss=dice_loss, metrics=[dice_loss, accuracy,recall, precision, IoU])

In [7]:
# Methods to add results of a patch   to    the total results of a larger area. 
#The operator could be min (useful if there are too many false positives), max (useful for tackle false negatives)
#res:mask [rows,cols] predition=np.squeeze(prediction[i], axis = -1) (col, row, wi, he) = batch_pos[i]
def addTOResult(res, prediction, row, col, he, wi,operator,ie_width):
    currValue = res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width]
    newPredictions = prediction
    
# IMPORTANT: MIN can't be used as long as the mask is initialed with 0!!!!! 
#If you want to use MIN initial the mask with -1 and handle the case of default value(-1) separately.
    if operator == 'min': # Takes the min of current prediction and new prediction for each pixel  
        currValue [currValue == -1] = 1 #Replace -1 with 1 in case of MIN  
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = np.minimum(currValue, newPredictions)
    elif operator == 'max':
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = np.maximum(currValue, newPredictions)
    else:
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = newPredictions  
    return (res)

In [8]:
# Methods that actually makes the predictions
def predict_using_model(model, batch, batch_pos, mask,operator,ie_width):
#     print(batch)
    tm = np.stack(batch, axis = 0)
#     print(tm)
    prediction = model.predict(tm)
    for i in range(len(batch_pos)): 
        (col, row, wi, he) = batch_pos[i]
        p = np.squeeze(prediction[i], axis = -1)
        # Instead of replacing the current values with new values, use the user specified operator (MIN,MAX,REPLACE)
        mask = addTOResult(mask, p, row, col, he, wi,operator,ie_width)  
    return mask

def detect_lake(fullPath, NDWI_img, bands_img, operator ,width=508, height=508, stride = 254,ie_width=92,bandNum=5):
    nols, nrows = NDWI_img.meta['width'], NDWI_img.meta['height']
    meta = NDWI_img.meta.copy() 
    if 'float' not in meta['dtype']: #The prediction is a float so we keep it as float to be consistent with the prediction. 
        meta['dtype'] = np.float32
    col_index=list(range(0, nols-width, stride))
    col_index.append(nols-width)
    row_index=list(range(0, nrows-height, stride))
    row_index.append(nrows-height)
    offsets = product(col_index,row_index)
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    print('the size of current NDWI_img',nrows, nols) 

    mask = np.zeros((nrows, nols), dtype=meta['dtype'])

#     mask = mask -1   # Note: The initial mask is initialized with -1 instead of zero   to handle the MIN case (see addToResult)
    batch = []
    batch_pos = [ ]
    for col_off, row_off in  tqdm(offsets):
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, NDWI_img.transform) 
#         hbh:notice datatype is float
        patch = np.full((height, width, bandNum),-1.0)#Add -1 padding in case of corner images
        if bandNum==1:
            read_ndwi_img=NDWI_img.read(window=window)
            temp_im = read_ndwi_img
        elif bandNum==3:
            read_ndwi_img=NDWI_img.read(window=window)/100
            read_bands_img = bands_img.read(window=window)/1000
            temp_im = np.concatenate((read_ndwi_img,np.expand_dims(read_bands_img[1],axis=0),np.expand_dims(read_bands_img[3],axis=0)), axis=0)
        else:#band==5
            read_ndwi_img=NDWI_img.read(window=window)/100
            read_bands_img = bands_img.read(window=window)/1000
            temp_im = np.concatenate((read_ndwi_img,read_bands_img), axis=0)
        temp_im =  np.transpose(temp_im, axes=(1,2,0))

        patch[:window.height, :window.width] = temp_im   
        batch.append(patch)
        batch_pos.append((window.col_off, window.row_off, window.width, window.height))
        
        if (len(batch) == config.BATCH_SIZE):
            mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
            batch = []
            batch_pos = []
            
    # To handle the edge of images as the image size may not be divisible by n complete batches and few frames on the edge may be left.
    if batch:
        mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
        batch = []
        batch_pos = []

    return(mask, meta)

In [9]:
def raster2vector(raster_path, vecter_path, field_name="class", ignore_values = None):
    
    # 读取路径中的栅格数据
    raster = gdal.Open(raster_path)
    # in_band 为想要转为矢量的波段,一般需要进行转矢量的栅格都是单波段分类结果
    # 若栅格为多波段,需要提前转换为单波段
    band = raster.GetRasterBand(1)
    
    # 读取栅格的投影信息,为后面生成的矢量赋予相同的投影信息
    prj = osr.SpatialReference()
    prj.ImportFromWkt(raster.GetProjection())
    
    
    drv = ogr.GetDriverByName("ESRI Shapefile")
    # 若文件已经存在,删除
    if os.path.exists(vecter_path):
        drv.DeleteDataSource(vecter_path)
    # 创建目标文件
    polygon = drv.CreateDataSource(vecter_path)
    # 创建面图层
    poly_layer = polygon.CreateLayer(vecter_path[:-4], srs=prj, geom_type=ogr.wkbMultiPolygon)
    # 添加浮点型字段,用来存储栅格的像素值
    field = ogr.FieldDefn(field_name, ogr.OFTReal)  
    poly_layer.CreateField(field)
    
    # FPolygonize将每个像元转成一个矩形，然后将相似的像元进行合并
    # 设置矢量图层中保存像元值的字段序号为0
    gdal.FPolygonize(band, None, poly_layer, 0)
    
    # 删除ignore_value链表中的类别要素
    if ignore_values is not None:
        for feature in poly_layer:
            class_value = feature.GetField('class')
            for ignore_value in ignore_values:
                if class_value==ignore_value:
                    # 通过FID删除要素
                    poly_layer.DeleteFeature(feature.GetFID())
                    break
                
    polygon.SyncToDisk()
    polygon = None
    print('Vector File Exported Successfully!')
        
def writeMaskToDisk(detected_mask, detected_meta, wp, write_as_type = 'uint8', th = 0.5, create_countors = False):
    # Convert to correct required before writing
    if 'float' in str(detected_meta['dtype']) and 'int' in write_as_type:
        print(f'Converting prediction from {detected_meta["dtype"]} to {write_as_type}, using threshold of {th}')#float32 to uint8
#         initial code have problem of big lake
        detected_mask[detected_mask<th]=0
        detected_mask[detected_mask>=th]=1
        detected_mask = detected_mask.astype(write_as_type)#'uint8'
        detected_meta['dtype'] =  write_as_type
    
    # compress tif
    detected_meta.update({"compress": 'lzw'})
    
    with rasterio.open(wp, 'w', **detected_meta) as outds:
        outds.write(detected_mask, 1)

In [10]:
# # Predict trees in the all the files in the input image dir 
# # Depending upon the available RAM, images may not to be split before running this cell.
# # Use the Auxiliary-2-SplitRasterToAnalyse if the images are too big to be analysed in memory.
# all_files = []
# for root, dirs, files in os.walk(config.input_image_dir):
#     for file in files:
#         if file.endswith(config.input_image_type)and file.startswith(config.ndwi_fn_st):
#              all_files.append((os.path.join(root, file), file))
# print(all_files)

# operator='max'
# suffix='_iew{}'.format(config.ignore_edge_width)
# for fullPath, filename in all_files:
#     outputFile = os.path.join(config.output_dir, filename.replace(config.ndwi_fn_st, config.output_prefix + operator + suffix) )
#     if not os.path.isfile(outputFile) or config.overwrite_analysed_files: 
#         with rasterio.open(fullPath) as NDWI:
#             with rasterio.open(fullPath.replace(config.ndwi_fn_st,config.bands_fn_st).replace('ndwi_int8','bands_int16')) as bands:
#                 print(fullPath)
#                 detectedMask, detectedMeta = detect_lake(fullPath,NDWI,bands,operator,width = config.WIDTH, height = config.HEIGHT, stride = config.STRIDE,ie_width=config.ignore_edge_width,bandNum=config.band_num)
#                 writeMaskToDisk(detectedMask, detectedMeta, outputFile, write_as_type = config.output_dtype, th = 0.5, create_countors = False)            
# #                 vecter_path =outputFile.replace('tif','shp')
# #                 field_name = "class"
# # #            第0类删除,若实际情况不需要1类和2类,则ignore_values = [1,2]
# #                 ignore_values = [0]
# #                 raster2vector(outputFile, vecter_path, field_name=field_name, ignore_values = ignore_values)#, ignore_values = ignore_values
#     else:
#         print('File already analysed!', fullPath)
        
# print('finish')

In [11]:
def detect_lake(fullPath, image, operator ,width=508, height=508,stride = 254,ie_width=92,bandNum=5):
    nols, nrows = image.meta['width'], image.meta['height']
    meta = image.meta.copy() 
    if 'float' not in meta['dtype']: #The prediction is a float so we keep it as float to be consistent with the prediction. 
        meta['dtype'] = np.float32
    meta['nodata']=None
    meta['count']=1
    col_index=list(range(0, nols-width, stride))
    if(nols-width)>0:
        col_index.append(nols-width)
    row_index=list(range(0, nrows-height, stride))
    if (nrows-height)>0:
        row_index.append(nrows-height)
    offsets = product(col_index,row_index)
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    print('the size of current image',nrows, nols) 

    mask = np.zeros((nrows, nols), dtype=meta['dtype'])

#     mask = mask -1   # Note: The initial mask is initialized with -1 instead of zero   to handle the MIN case (see addToResult)
    batch = []
    batch_pos = [ ]
    for col_off, row_off in  tqdm(offsets):
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, image.transform) 
#         hbh:notice datatype is float
        patch = np.full((height, width, bandNum),-1.0)#Add -1 padding in case of corner images
        read_image=image.read(window=window)
        temp_im = read_image/1000
        temp_im =  np.transpose(temp_im, axes=(1,2,0))

        patch[:window.height, :window.width] = temp_im   
        batch.append(patch)
        batch_pos.append((window.col_off, window.row_off, window.width, window.height))
        
        if (len(batch) == config.BATCH_SIZE):
            mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
            batch = []
            batch_pos = []
            
    # To handle the edge of images as the image size may not be divisible by n complete batches and few frames on the edge may be left.
    if batch:
        mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
        batch = []
        batch_pos = []

    return(mask, meta)

In [19]:
# Predict trees in the all the files in the input image dir 
# Depending upon the available RAM, images may not to be split before running this cell.
# Use the Auxiliary-2-SplitRasterToAnalyse if the images are too big to be analysed in memory.
all_files = []
for root, dirs, files in os.walk(r'D:\lakemapping\2_dataset\patchesReshape/test'):
    for file in files:
        if file.endswith(config.input_image_type)and file.startswith('pad_image'):
             all_files.append((os.path.join(root, file), file))
print(all_files)

operator='max'
suffix='_test'
for fullPath, filename in all_files:
    outputFile = os.path.join(r'D:\lakemapping\4_prediction\sample747\model_0108_572/test', filename.replace('pad_image', config.output_prefix + operator + suffix) )
    if not os.path.isfile(outputFile) or config.overwrite_analysed_files: 
        with rasterio.open(fullPath) as image:
            print(fullPath)
            detectedMask, detectedMeta = detect_lake(fullPath,image,operator,width = config.WIDTH, height = config.HEIGHT, stride = config.STRIDE,ie_width=config.ignore_edge_width,bandNum=config.band_num)
            writeMaskToDisk(detectedMask, detectedMeta, outputFile, write_as_type = config.output_dtype, th = 0.5, create_countors = False)            
            vecter_path =outputFile.replace('tif','shp')
            field_name = "class"
#            第0类删除,若实际情况不需要1类和2类,则ignore_values = [1,2]
            ignore_values = [0]
            raster2vector(outputFile, vecter_path, field_name=field_name, ignore_values = ignore_values)#, ignore_values = ignore_values
    else:
        print('File already analysed!', fullPath)
        
print('finish')

[('D:\\lakemapping\\2_dataset\\patchesReshape/test\\type0\\pad_image_10031.tif', 'pad_image_10031.tif'), ('D:\\lakemapping\\2_dataset\\patchesReshape/test\\type0\\pad_image_10892.tif', 'pad_image_10892.tif'), ('D:\\lakemapping\\2_dataset\\patchesReshape/test\\type0\\pad_image_12351.tif', 'pad_image_12351.tif'), ('D:\\lakemapping\\2_dataset\\patchesReshape/test\\type0\\pad_image_12666.tif', 'pad_image_12666.tif'), ('D:\\lakemapping\\2_dataset\\patchesReshape/test\\type0\\pad_image_13177.tif', 'pad_image_13177.tif'), ('D:\\lakemapping\\2_dataset\\patchesReshape/test\\type0\\pad_image_13618.tif', 'pad_image_13618.tif'), ('D:\\lakemapping\\2_dataset\\patchesReshape/test\\type0\\pad_image_13979.tif', 'pad_image_13979.tif'), ('D:\\lakemapping\\2_dataset\\patchesReshape/test\\type0\\pad_image_14006.tif', 'pad_image_14006.tif'), ('D:\\lakemapping\\2_dataset\\patchesReshape/test\\type0\\pad_image_14111.tif', 'pad_image_14111.tif'), ('D:\\lakemapping\\2_dataset\\patchesReshape/test\\type0\\pad_i

15it [00:00, 80.37it/s]


1/1 [==============================] - 0s 89ms/step
Converting prediction from <class 'numpy.float32'> to uint8, using threshold of 0.5
Vector File Exported Successfully!
D:\lakemapping\2_dataset\patchesReshape/test\type0\pad_image_10892.tif
the size of current image 3017 3101


13it [00:00, 63.80it/s]

1/1 [==============================] - 2s 2s/step


...

1/1 [==============================] - 0s 49ms/step


27it [00:00, 31.18it/s]

1/1 [==============================] - 0s 45ms/step


42it [00:01, 31.01it/s]


1/1 [==============================] - 0s 34ms/step
Converting prediction from <class 'numpy.float32'> to uint8, using threshold of 0.5
Vector File Exported Successfully!
finish


In [18]:
# outputFile=r'/media/nkd/backup/5_lakemapping/iew100/predition_b5_max_iew50_4976.tif'
# vecter_path =outputFile.replace('tif','shp')
# field_name = "class"
# #            第0类删除,若实际情况不需要1类和2类,则ignore_values = [1,2]
# ignore_values = [0]
# raster2vector(outputFile, vecter_path, field_name=field_name, ignore_values = ignore_values)#, ignore_values = ignore_values